---
# Fall 2020 MATH 662 Presentation

---

Name: **Matt Ryan**

Acknowledgements:

[Fundamentals of Numerical Computation](https://fncbook.github.io/fnc/frontmatter.html) by Tobin A. Driscoll and Richard J. Braun

[Newton's law of universal gravitation](https://en.wikipedia.org/wiki/Newton%27s_law_of_universal_gravitation) From Wikipedia, the free encyclopedia

[Newton's laws of motion](https://en.wikipedia.org/wiki/Newton%27s_laws_of_motion) From Wikipedia, the free encyclopedia

---
## Topic: Runge-Kutta RK4 Method Applied to the Three-Body Problem
---
**What will be covered:**

- Euler's method
- Two-body problem - planet orbiting star
- Two-body problem - binary star system
- Runge-Kutta theory
- RK4 method
- RK4 applied to two-body problem
- RK4 applied to three-body problem in two dimensions

By the end of this presentation you should be able to use the RK4 method to solve a system of second order ordinary differential equations.

## Euler's Method

Euler's method is a simple way of approximating first order differential equations with initial conditions.  A first order ODE is defined as

$$u'(t) = f(t, u(t))$$
$$u(a) = u_0$$

for $a \le t \le b.$  By plugging the initial condition into $u'(a) = f(a, u(a))$, we know the slope of $u(t)$ at $t = a$.  We can use this to approximate $u(a+h)$, where h is called the step size:

$$u(a+h) = u_0 + h f(a, u(a)).$$

We now have an approximation for $u(a+h)$. We can repeat the process to get successive approximations for $u(t)$:

$$u(t_{i+1}) = u(t_i) + hf(t_i, u(t_i))$$

where $t_i = a + hi$, $h = \frac{b-a}{n}$, and $i = 0,\dots,n$.

### Example: Two-body problem - planet orbiting star

---
We can model a planet orbiting a star as a second order ODE using laws of physics described by Isaac Newton.


Newton's Law of Universal Gravitation:
$$F = G\frac{m_1m_2}{r^2}.$$

Newton's Second Law of Motion:
$$F = ma$$

Newton's Third Law of Motion:
$$F_{12} = -F_{21}$$

- $F$ is the force applied to one object by the other object.

- $G$ is the gravitational constant.

- $m_1$ and $m_2$ are the respective masses of two objects.

- $r$ is the distance between the two objects.

We want to vectorize the law of gravitation so direction matters.  Let's let $r$ be a two dimensional vector in the $xy$-plane and $\hat{r}$ be the directional unit vector.  Then the force on object 1 from object 2 is defined as

$$F_{12} = G\frac{m_1m_2}{\left|r\right|^2}\hat{r}$$

We can rewrite the unit vector as $\hat{r} = \frac{r}{|r|}$, where $|r|$ is the Euclidean norm of $r.$  Then

$$F_{12} = G\frac{m_1m_2}{\left|r\right|^3}r$$

and from the third law of motion

$$F_{21} = -G\frac{m_1m_2}{\left|r\right|^3}r.$$

We currently have equations for relating the positions of the objects to the force applied to each object.  However, we would like to rewrite these equations in terms of acceleration, which is the second derivative of position.  To do so, we simply divide the force by the corresponding mass.  Therefore

$$a_1 = \frac{F_{12}}{m_1} = G\frac{m_2}{\left|r\right|^3}r$$

and

$$a_2 = \frac{F_{21}}{m_2} = -G\frac{m_1}{\left|r\right|^3}r.$$

For the case of a planet orbiting a star, the mass of the planet is usually so small in comparison to the mass of the star that we can assume there is no force applied to the star by the planet.  This is not necessarily always true, but for our purposes we will accept this.  We can fix the star at the coordinate $(0,0)$ and that way $r_2$ = $r$.  So our second order ODE for the orbit of the planet becomes

$$\ddot{r} = -G\frac{m_1}{\left|r\right|^3}r.$$

But, didn't I say that Euler's method is used to approximate FIRST order ODEs?  Fear not - any second order ODE can be rewritten as a first order system.  Remember that velocity $\dot{r} = v$ and $\dot{v} = a, $ where $v$ is velocity and $a$ is acceleration.  The second order system of the motion of the planet is

$$\dot{r} = v$$
$$\dot{v} = -G\frac{m_1}{\left|r\right|^3}r.$$

We are now ready to apply Euler's method!

In [ ]:
import LinearAlgebra: norm
import Plots: plot, plot!, plot3d, plot3d!

In [ ]:
# Euler two-body - planetary orbit

G = 10  # gravitational constant
m = 10  # mass of star

h = .001     # step size
n = 500000  # iterations

r = [15, 0]  # initial position
v = [0, 3]  # initial velocity

rr = zeros(2,n)  # solution matrix

rr[:,1] = r  # save initial condition

for i=2:n
    a = -G*m*r/norm(r)^3  # calculate acceleration
    r = r + h*v  # update position using Euler formula
    v = v + h*a  # update velocity using Euler formula
    rr[:,i] = r  # save solution
end

In [ ]:
plot(rr[1,:], rr[2,:], label="Planet orbit")

In [ ]:
tt = [h*i for i=1:n]
plot3d(tt, rr[1,:], rr[2,:], label="Planet orbit")

---
### Example: Two-body problem - binary star system

---

The mass of object 2 is now large enough that the force applied to object 1 is no longer negligible, so we cannot assume that either body is stationary.

Taking this into consideration, all we need to do is include our second order equation for object 1:

$$\ddot{r_1} = G\frac{m_2}{\left|r\right|^3}r$$

$$\ddot{r_2} = -G\frac{m_1}{\left|r\right|^3}r.$$

The result is a first order system of 4 equations:

$$\dot{r_1} = v_1$$

$$\dot{r_2} = v_2$$

$$\dot{v_1} = G\frac{m_2}{\left|r\right|^3}r$$

$$\dot{v_2} = -G\frac{m_2}{\left|r\right|^3}r$$

In [ ]:
import LinearAlgebra: norm
import Plots: plot, plot!, plot3d, plot3d!

# Euler two-body - binary star system
G = 10
m1 = m2 = 10
r1 = [-5,0]  # posotion of object 1
r2 = [5,0]   # position of object 2
v1 = [0,-2]  # velocity of object 1
v2 = [0,2]   # velocity of object 2

h = .0005
n = 200000

rr1 = zeros(2,n)
rr2 = zeros(2,n)

rr1[:,1] = r1
rr2[:,1] = r2

for i=2:n
    r = r2 - r1
    F = G*m1*m2*r/norm(r)^3
    a1 = F / m1
    a2 = -F / m2

    # update position
    r1 += h*v1
    r2 += h*v2

    # update velocity
    v1 += h*a1
    v2 += h*a2

    rr1[:,i] = r1
    rr2[:,i] = r2
end

In [ ]:
plot(rr1[1,:], rr1[2,:])
plot!(rr2[1,:], rr2[2,:],color=2)

In [ ]:
tt = [h*i for i=1:n]
plot3d(tt, rr1[1,:], rr1[2,:])
plot3d!(tt, rr2[1,:], rr2[2,:],color=2)

---
## Runge-Kutta Methods

Clearly we need a better approximation for our solution to these problems.  While the Euler method works for very small values of h, the error grows too quickly to be practical.  Let's look at the Taylor series expansion of the solution to $u' = f(t, u)$:

$$\hat{u}(t_{i+1}) = \hat{u}(t_i) + h\hat{u}'(t_i) + \frac{1}{2}h^2\hat{u}''(t_i) + O(h^3).$$

Notice that the first two terms of this equation gives us Euler's method!  Euler's method is a first order Runge-Kutta method.  The error for the Euler method is in $O(h^2)$.  We would like to find an approximation with a smaller error term.  This means we need a way to compute $\hat{u}''$.  We can apply the multidimensional chain rule to get the following:

$$\hat{u}'' = f' = \frac{df}{dt} = \frac{\partial f}{\partial t} + \frac{\partial f}{\partial u}\frac{du}{dt} = f_t + f_uf.$$

By substituting this into the previous equation, we get

$$\hat{u}(t_{i+1}) = \hat{u}(t_i) + h\left[f(t_i,\hat{u}(t_i)) + \frac{h}{2}f_t(t_i,\hat{u}(t_i)) + \frac{h}{2}f(t_i,\hat{u}(t_i))f_u(t_i,\hat{u}(t_i))\right] + O(h^3). \quad (6.4.2)$$

We don't want to actually compute these partial derivatives.  We also know that

$$f(t_i+\alpha,\hat{u}(t_i) + \beta) = f(t_i,\hat{u}(t_i)) + \alpha f_t(t_i,\hat{u}(t_i)) + \beta f_u(t_i,\hat{u}(t_i)) + O(\alpha^2+|\alpha\beta| + \beta^2). \quad (6.4.3)$$

By equating this to the term in brackets, we find that $\alpha = \frac{h}{2}$ and $\beta = \frac{h}{2}f(t_i,\hat{u}(t_i))$.  We can substitute the left hand side of 6.4.3 into 6.4.2 using these values for $\alpha$ and $\beta$ to give us our new second order RK method:

$$u_{i+1} = u_i + hf(t_i+\frac{1}{2}h,u_i+\frac{1}{2}hf(t_i,u_i)).$$

This is called the improved Euler method, and is a second order RK method.  Notice that f is computed twice in this method.  We can break it up into two stages, $k_1$ and $k_2$, where

$$k_1 = hf(t_i,u_i),$$

$$k_2 = hf(t_i+\frac{1}{2}h,u_i+\frac{1}{2}k_1),$$
and
$$u_{i+1} = u_i + k_2.$$

The idea here is that we use the slope from the first stage to take a half time step.  We then evaluate the slope again at at this half step, and use the new slope to approximate the solution over the whole step.

This second order approximation is definitely an improvement over the first order Euler method, but we would like to use a fourth order method with an error of $O(h^5)$.  This method is called RK4, and is one of the most well known formulas for solving numerical differential equations.

To do this we will need an approximation of $\hat{u}'''$ and $\hat{u}^{(4)}$.  I will not derive this directly but the derivation is the same as the derivation for the Improved Euler, just using more terms.  The complexity of the algebra increases drastically, but RK methods of much higher orders have been found.

As the Improved Euler method has two stages, RK4 has four:

$$k_1 = hf(t_i,u_i)$$

$$k_2 = hf(t_i+\frac{h}{2},u_i+\frac{1}{2}k_1)$$

$$k_3 = hf(t_i+\frac{h}{2},u_i+\frac{1}{2}k_2)$$

$$k_4 = hf(t_i+h,u_i+k_3)$$
and
$$u_{i+1} = u_i + \frac{1}{6}k_1 + \frac{1}{3}k_2 + \frac{1}{3}k_3 + \frac{1}{6}k_4.$$

The coefficients act as a sort of weighted average of the four slopes, and are not at all random.  They come directly from the derivation.

Here is an image that describes what is really going on here:

<img src="files/Runge-Kutta_slopes.svg">

By HilberTraum - Own work, CC BY-SA 4.0, https://commons.wikimedia.org/w/index.php?curid=64366870

---
### Example: Two-body problem using RK4 - binary star system
---

In [ ]:
G = 10
m1 = m2 = 10

# one iteration of rk4
function rk4(f, u, h)
    k1 = h*f(u)
    k2 = h*f(u + k1/2)
    k3 = h*f(u + k2/2)
    k4 = h*f(u + k3)
    return u + k1/6 + k2/3 + k3/3 + k4/6
end


# acceleration
function a(r1, r2)
    r = r2 - r1
    r_norm = norm(r)
    F = G*m1*m2*r/r_norm^3
    a1 = F / m1
    a2 = -F / m2
    return a1, a2
end


function f(u)
    r1, r2, v1, v2 = u
    a1, a2 = a(r1, r2)
    return [v1, v2, a1, a2]
end

In [ ]:
# two body rk4
import LinearAlgebra: norm
import Plots: plot, plot!, plot3d, plot3d!

# initial conditions
r1 = [-5,0]
r2 = [5,0]
v1 = [0,-2]
v2 = [0,2]

h = .1    # step size
n = 5000  # steps

# solution matrices
rr1 = zeros(2,n)
rr2 = zeros(2,n)

rr1[:,1] = r1
rr2[:,1] = r2

u = [r1, r2, v1, v2]

for i=2:n
    u = rk4(f, u, h)
    rr1[:,i] = u[1]
    rr2[:,i] = u[2]
end

In [ ]:
plot(rr1[1,:], rr1[2,:])
plot!(rr2[1,:], rr2[2,:], color=2)

In [ ]:
tt = [h*i for i=1:n]
plot3d(tt, rr1[1,:], rr1[2,:])
plot3d!(tt, rr2[1,:], rr2[2,:], color=2)

In [ ]:
# three body rk4
import LinearAlgebra: norm
import Plots: plot, plot!, plot3d, plot3d!

G = 10
m1 = m2 = 10

function rk4(f, t, u, h)
    k1 = h*f(t, u)
    k2 = h*f(t + h/2, u + k1/2)
    k3 = h*f(t + h/2, u + k2/2)
    k4 = h*f(t + h, u + k3)
    return u + k1/6 + k2/3 + k3/3 + k4/6
end

function a(r, m)
    return G*m*r/norm(r)^3
end

function f(t, u)
    r1, r2, r3, v1, v2, v3 = u
    a1 = a(r2-r1, m2)
    a2 = a(r1-r2, m1)
    a3 = a(r1-r3, m1) + a(r2-r3, m2)
    return [v1, v2, v3, a1, a2, a3]
end

h = .1
n = 5000
t = 0

# initial conditions
r1 = [-5, 0]
r2 = [5, 0]
r3 = [50, 0]
v1 = [0, -2]
v2 = [0, 2]
v3 = [0, 2]

u = [r1, r2, r3, v1, v2, v3]

# solution matrices
rr1 = [r1 zeros(2,n)]
rr2 = [r2 zeros(2,n)]
rr3 = [r3 zeros(2,n)]

for i=1:n
    u = rk4(f, t, u, h)
    t += h
    rr1[:,i+1] = u[1]
    rr2[:,i+1] = u[2]
    rr3[:,i+1] = u[3]
end

In [ ]:
plot(rr1[1,:], rr1[2,:], label="Star 1", legend=:topright, xlabel="x", ylabel="y")
plot!(rr2[1,:], rr2[2,:], label="Star 2", color=2)
plot!(rr3[1,:], rr3[2,:], label="Planet", color=3)

In [ ]:
tt = [i*h for i=0:n]
plot3d(tt, rr1[1,:], rr1[2,:], label="Star 1", xlabel="Time", ylabel="x", zlabel="y")
plot3d!(tt, rr2[1,:], rr2[2,:], label="Star 2", color=2)
plot3d!(tt, rr3[1,:], rr3[2,:], label="Planet", color=3)